# LSTM
Using LSTM to predict price

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn import preprocessing
import matplotlib.pyplot as plt

D:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Process the train and test data

In [2]:
def readTrain(filename = 'train_data.csv'):
	df = pd.read_csv(filename)
	train_data = df.iloc[:,3:10]
	return train_data

In [3]:
Train = readTrain()
diff_price = []
_Train = Train.values

In [4]:
def get_diff_volume(_Train):
    diff_volume = []
    for i in range(len(Train)-1):
        diff_volume.append(_Train[i+1,2] - _Train[i,2])
    diff_volume.append(diff_volume[-1])
    for i in range(len(diff_volume)):
        if(diff_volume[i]<0):
            diff_volume[i] = 0
    return np.array(diff_volume)
Train['diff_volume'] = get_diff_volume(_Train)

In [22]:
def get_label(_Train):
    label = []
    for i in range(len(_Train) - 20 -10):
        label.append(_Train[i+10:i+30,0].mean())
    return np.array(label)
Label = get_label(_Train)

In [23]:
Train

,MidPrice,LastPrice,Volume,BidPrice1,BidVolume1,AskPrice1,AskVolume1,diff_volume
0,3.7865,3.786,1081100.0,3.786,21500.0,3.787,12900.0,344800.0
1,3.7835,3.784,1425900.0,3.783,44200.0,3.784,119200.0,59600.0
2,3.7835,3.784,1485500.0,3.783,42700.0,3.784,61100.0,66100.0
3,3.7845,3.785,1551600.0,3.784,145900.0,3.785,16900.0,511700.0
4,3.7835,3.784,2063300.0,3.783,347700.0,3.784,200100.0,345300.0
5,3.7835,3.784,2408600.0,3.783,352600.0,3.784,70700.0,86200.0
6,3.7860,3.785,2494800.0,3.784,415000.0,3.788,5800.0,0.0
7,3.7860,3.785,2494800.0,3.784,133200.0,3.788,85900.0,116700.0
8,3.7860,3.784,2611500.0,3.784,16500.0,3.788,85900.0,7700.0
9,3.7855,3.784,2619200.0,3.784,8800.0,3.787,80100.0,0.0


In [24]:
X_scaler = preprocessing.MinMaxScaler()
train_data = X_scaler.fit_transform(Train)
train_data

array([[0.86437908, 0.86437908, 0.00180071, ..., 0.86437908, 0.00142368,
        0.04231506],
       [0.85947712, 0.86111111, 0.00252335, ..., 0.85947712, 0.01316164,
        0.00731432],
       [0.85947712, 0.86111111, 0.00264826, ..., 0.85947712, 0.00674607,
        0.00811202],
       ...,
       [0.41013072, 0.41176471, 0.63062417, ..., 0.41013072, 0.10282293,
        0.11438822],
       [0.41176471, 0.41176471, 0.63257763, ..., 0.41176471, 0.0393429 ,
        0.0262897 ],
       [0.41013072, 0.41176471, 0.63302659, ..., 0.41013072, 0.03376445,
        0.0262897 ]])

In [25]:
Label = np.reshape(Label,(-1,1))
Y_scaler = preprocessing.MinMaxScaler()
y_train_data = Y_scaler.fit_transform(Label)
print(y_train_data.shape)
y_train_data

(430009, 1)


array([[0.86291049],
       [0.86286954],
       [0.86266481],
       ...,
       [0.40926214],
       [0.40934403],
       [0.40950782]])

In [26]:
train_data.shape

(430039, 8)

In [27]:
def buildTrain(Train_data, pastDay=10, futureDay=20):
	x_train = []
	for i in range(Train_data.shape[0]-futureDay-pastDay):
		x_train.append(Train_data[i:i+pastDay,:])
	return np.array(x_train)
X_train = buildTrain(train_data)
print(X_train.shape)

(430009, 10, 8)


In [28]:
X_train

array([[[8.64379085e-01, 8.64379085e-01, 1.80071381e-03, ...,
         8.64379085e-01, 1.42368238e-03, 4.23150557e-02],
        [8.59477124e-01, 8.61111111e-01, 2.52334663e-03, ...,
         8.59477124e-01, 1.31616360e-02, 7.31431938e-03],
        [8.59477124e-01, 8.61111111e-01, 2.64825648e-03, ...,
         8.59477124e-01, 6.74606588e-03, 8.11202199e-03],
        ...,
        [8.63562092e-01, 8.62745098e-01, 4.76355030e-03, ...,
         8.66013072e-01, 9.48455366e-03, 1.43218301e-02],
        [8.63562092e-01, 8.61111111e-01, 5.00813049e-03, ...,
         8.66013072e-01, 9.48455366e-03, 9.44970792e-04],
        [8.62745098e-01, 8.61111111e-01, 5.02426817e-03, ...,
         8.64379085e-01, 8.84410087e-03, 0.00000000e+00]],

       [[8.59477124e-01, 8.61111111e-01, 2.52334663e-03, ...,
         8.59477124e-01, 1.31616360e-02, 7.31431938e-03],
        [8.59477124e-01, 8.61111111e-01, 2.64825648e-03, ...,
         8.59477124e-01, 6.74606588e-03, 8.11202199e-03],
        [8.61111111e-01, 

array([0.62637363, 0.62225275, 0.61342229, ..., 0.63441915, 0.63481162,
       0.63559655])

In [29]:
def shuffle(X,Y):
	randomList = np.arange(X.shape[0])
	np.random.shuffle(randomList)
	return X[randomList], Y[randomList]

# shuffle the data, and random seed is 10
X_train, Y_train = shuffle(X_train, y_train_data)

In [30]:
def splitData(X,Y,rate):
	X_train = X[int(X.shape[0]*rate):]
	Y_train = Y[int(Y.shape[0]*rate):]
	X_val = X[:int(X.shape[0]*rate)]
	Y_val = Y[:int(Y.shape[0]*rate)]
	return X_train, Y_train, X_val, Y_val
# split training data and validation data
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

In [47]:
from keras import backend as K

def loss_func(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true - y_pred)))
def buildManyToOneModel(shape):
	model = Sequential()
	model.add(LSTM(128, input_length=shape[1], input_dim=shape[2],activation = 'sigmoid'))
	#model.add(LSTM(128, activation='tanh',input_length=shape[1], input_dim=shape[2],return_sequences=True))
	#model.add(LSTM(128, activation='tanh',return_sequences = False))
	# output shape: (1, 1)
	model.add(Dense(128, activation='sigmoid'))
	model.add(Dense(1,activation='sigmoid'))
	model.compile(loss="mse", optimizer="adam")
	model.summary()
	return model

In [48]:
model = buildManyToOneModel(X_train.shape)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  import sys
D:\anaconda\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, activation="sigmoid", input_shape=(10, 8))`
  import sys


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 128)               70144     
_________________________________________________________________
dense_10 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 129       
Total params: 86,785
Trainable params: 86,785
Non-trainable params: 0
_________________________________________________________________


In [49]:
from keras import optimizers
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

In [50]:
def loss_fun(y_true, y_pred):
    y_true = y_true * scaler.scale_[-1] + scaler.mean_[-1]
    y_pred = y_pred * scaler.scale_[-1] + scaler.mean_[-1]
    return K.mean(K.square(y_true - y_pred))

In [52]:
# model = load_model('double_LSTM_preprocess.h5')
model.compile(loss="mse", optimizer=adam)
callback = EarlyStopping(monitor='loss', patience=3, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=10, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback], shuffle = True)
model.save('double_LSTM_preprocess_v2.h5')

Train on 387009 samples, validate on 43000 samples
Epoch 1/10
387009/387009 [==============================] - 77s 199us/step - loss: 0.0013 - val_loss: 2.4034e-05
Epoch 2/10
387009/387009 [==============================] - 70s 182us/step - loss: 2.5024e-05 - val_loss: 9.4246e-05
Epoch 3/10
387009/387009 [==============================] - 71s 185us/step - loss: 1.8558e-05 - val_loss: 5.1424e-05
Epoch 4/10
387009/387009 [==============================] - 76s 196us/step - loss: 1.6157e-05 - val_loss: 1.0009e-05
Epoch 5/10
387009/387009 [==============================] - 70s 180us/step - loss: 1.4520e-05 - val_loss: 3.4673e-05
Epoch 6/10
387009/387009 [==============================] - 70s 180us/step - loss: 1.3599e-05 - val_loss: 2.0721e-05
Epoch 7/10
387009/387009 [==============================] - 70s 180us/step - loss: 1.2436e-05 - val_loss: 1.0275e-05
Epoch 8/10
387009/387009 [==============================] - 70s 181us/step - loss: 1.2232e-05 - val_loss: 9.0283e-06
Epoch 9/10
387009

In [53]:
# model.save('double_LSTM_preprocess.h5')
from keras.models import load_model

In [54]:
# model = load_model('double_LSTM_v2.h5')
test_X_pd = readTrain('test_data.csv').iloc[1420:,:]
_test_X_pd = test_X_pd.values
test_diff_volume = []
for i in range(len(_test_X_pd)-1):
    test_diff_volume.append(_test_X_pd[i+1,2] - _test_X_pd[i,2])
test_diff_volume.append(test_diff_volume[-1])
for i in range(len(test_diff_volume)):
    if(test_diff_volume[i]<0):
        test_diff_volume[i] = 0
test_X_pd['diff_volume'] = np.array(test_diff_volume)

In [55]:
test_X_pd

,MidPrice,LastPrice,Volume,BidPrice1,BidVolume1,AskPrice1,AskVolume1,diff_volume
1420,3.3595,3.359,4764000.0,3.359,574500.0,3.360,62900.0,38200.0
1421,3.3595,3.359,4802200.0,3.359,543300.0,3.360,63900.0,37700.0
1422,3.3595,3.359,4839900.0,3.359,610600.0,3.360,68700.0,197100.0
1423,3.3595,3.360,5037000.0,3.359,611100.0,3.360,471100.0,397800.0
1424,3.3595,3.360,5434800.0,3.359,280000.0,3.360,424800.0,303300.0
1425,3.3585,3.359,5738100.0,3.358,3800.0,3.359,360800.0,20400.0
1426,3.3585,3.359,5758500.0,3.358,3800.0,3.359,56800.0,4100.0
1427,3.3570,3.358,5762600.0,3.356,1012300.0,3.358,86300.0,16200.0
1428,3.3570,3.358,5778800.0,3.356,105400.0,3.358,86000.0,125200.0
1429,3.3555,3.356,5904000.0,3.355,668900.0,3.356,196400.0,10257400.0


In [56]:
test_X = X_scaler.transform(test_X_pd)
test_X = np.reshape(test_X,(-1,10,8))
prediction = model.predict(test_X)
prediction = Y_scaler.inverse_transform(prediction)

In [57]:
prediction = np.reshape(prediction,(-1))
prediction

array([3.3591595, 3.3648357, 3.3554688, 3.3540893, 3.3513138, 3.3516262,
       3.346258 , 3.3484848, 3.3589182, 3.3591588, 3.359105 , 3.3616796,
       3.357637 , 3.3594756, 3.36886  , 3.3640783, 3.365374 , 3.3629682,
       3.3569803, 3.3595932, 3.3548667, 3.353631 , 3.3541725, 3.3562148,
       3.3564303, 3.361398 , 3.3580968, 3.3515317, 3.3496356, 3.346065 ,
       3.3439333, 3.345262 , 3.3449612, 3.3413887, 3.3441005, 3.3478048,
       3.3473   , 3.348517 , 3.3439693, 3.3394885, 3.3446977, 3.347204 ,
       3.3491268, 3.3484912, 3.3591118, 3.3623757, 3.3653438, 3.3601387,
       3.3609815, 3.3593466, 3.3582044, 3.3604405, 3.35888  , 3.3637466,
       3.357402 , 3.3561647, 3.354483 , 3.3535686, 3.3512373, 3.3519082,
       3.3525898, 3.3520813, 3.3507197, 3.3523343, 3.3516462, 3.354013 ,
       3.3535037, 3.353568 , 3.3476312, 3.3477507, 3.3472795, 3.3495755,
       3.3493116, 3.3482144, 3.3461208, 3.3450663, 3.345535 , 3.3451273,
       3.3414059, 3.3431308, 3.3432314, 3.3414905, 

array([3.355664 , 3.364747 , 3.3557255, 3.3564796, 3.3526108, 3.3529894,
       3.3483384, 3.3508835, 3.3608932, 3.3596919, 3.3612561, 3.3613012,
       3.3595557, 3.3600843, 3.37056  , 3.364882 , 3.3663452, 3.3643515,
       3.3569424, 3.3596933, 3.3569882, 3.355061 , 3.3562307, 3.3584976,
       3.3581681, 3.3637345, 3.3592174, 3.3531916, 3.3510745, 3.3485527,
       3.3460433, 3.3474445, 3.3463442, 3.3431516, 3.345902 , 3.3488898,
       3.3495312, 3.3506176, 3.3456657, 3.3410542, 3.3470855, 3.349346 ,
       3.351245 , 3.350912 , 3.3614051, 3.3649228, 3.3672318, 3.3621683,
       3.3632758, 3.3609617, 3.3598983, 3.3631322, 3.3610342, 3.3658772,
       3.3595066, 3.3587234, 3.356562 , 3.3554125, 3.3531578, 3.3539627,
       3.3542593, 3.3536649, 3.3524482, 3.3545222, 3.3531156, 3.3559866,
       3.3554406, 3.3556268, 3.349832 , 3.3491912, 3.3492994, 3.3520358,
       3.3512292, 3.349147 , 3.3481894, 3.3469594, 3.3481033, 3.3459988,
       3.3435023, 3.3451526, 3.3451679, 3.3435218, 

In [58]:
caseid = list(range(143,1001))
df = pd.DataFrame({'caseid':caseid, 'midprice':prediction})
df.to_csv('double_LSTM_Dense_after_normal.csv',index = False, sep = ',')